In [1]:
def get_pipeline(vuelo, zona, tipo):
    return f'''{{
      "pipeline":[
        {{
          "type":"readers.pgpointcloud",
          "connection":"host='172.17.0.2' dbname='pointclouds' user='postgres' password='ventanuco' port='5432'",
          "table":"{vuelo}_zona{zona}_clas",
          "column": "pa",
          "spatialreference":"EPSG:25830",
          "where": "pc_numpoints(pa) is not null"
        }},
        {{
          "type": "filters.range",
          "limits": "Classification[1:]"
        }},
        {{
          "type":"filters.hag_dem",
          "raster": "MDT/MDT02-ETRS89-HU30-0174-2-COB2.tif"
        }},
        {{
          "type": "filters.range",
          "limits": "HeightAboveGround{tipo['range']}"
        }},
        {{
          "type":"writers.gdal",
          "filename":"FINALES/{vuelo}_zona{zona}_{tipo['nombre']}.tif",
          "gdaldriver":"GTiff",
          "output_type":"max",
          "resolution":"1.0",
          "dimension":"HeightAboveGround"
        }}
      ]
    }}'''

In [ ]:
import pdal

for vuelo in ['vuelo2']:
    for zona in [2]:
        for tipo in [{'nombre': 'arbolado', 'range': '[3:]'}, {'nombre': 'matorral', 'range': '[0.2:3]'}]:
            pipeline = pdal.Pipeline(get_pipeline(vuelo, zona, tipo))
            pipeline.execute()